In [1]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()
#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset6.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[1,2])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)
ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=17, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1                             

Instructions for updating:
non-resource variables are not supported in the long term
Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.104859
                      0.241841
                      0.476699
                      0.281853
                      0.792169
                      0.120836
                      0.399621
                      0.452736
                      0.115953
                      0.058893
                      0.209886
                      0.051525
                      0.041965
                      0.199646
                      0.292161
                      0.083829
                      0.508873
 DT - feature_importance_vals
                     0.003128
                     0.068357
                     0.127512
                     0.050683
                     0.017416
                     0.000000
                     0.154177
                     0.045905
                     0.002902
                     0.007681
                     0.011373
                     0.000000
                     0.006662
                     0

keras is no longer supported, please use tf.keras instead.


 NN - feature_importance_vals
                     0.016929
                     0.056092
                     0.173334
                     0.136472
                     0.038971
                     0.013763
                     0.042720
                     0.039395
                     0.017335
                     0.030099
                     0.013848
                     0.010919
                     0.008500
                     0.037430
                     0.043572
                     0.018706
                     0.082652
Iteration 2


...

 XGB - feature_importance_vals
                      0.091768
                      0.302222
                      0.445957
                      0.234923
                      0.784633
                      0.107935
                      0.372821
                      0.481040
                      0.112237
                      0.048114
                      0.238213
                      0.064754
                      0.015972
                      0.247045
                      0.287811
                      0.036409
                      0.524022
 DT - feature_importance_vals
                     0.002801
                     0.048362
                     0.109508
                     0.065691
                     0.135503
                     0.001599
                     0.048608
                     0.037250
                     0.003816
                     0.022476
                     0.012375
                     0.000651
                     0.005787
                     0

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.107601
                      0.297081
                      0.465855
                      0.242993
                      0.896141
                      0.110834
                      0.270440
                      0.448042
                      0.108624
                      0.074776
                      0.232812
                      0.059010
                      0.024385
                      0.203933
                      0.292116
                      0.066642
                      0.528535
 DT - feature_importance_vals
                     0.002532
                     0.048077
                     0.097151
                     0.065685
                     0.027736
                     0.001923
                     0.152340
                     0.045549
                     0.004546
                     0.006186
                     0.005624
                     0.002868
                     0.009334
                     0

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.106811
                      0.294136
                      0.449155
                      0.245240
                      0.789992
                      0.105516
                      0.381356
                      0.469506
                      0.102245
                      0.042685
                      0.247501
                      0.059832
                      0.034284
                      0.239879
                      0.282484
                      0.046907
                      0.541370
 DT - feature_importance_vals
                     0.003331
                     0.045122
                     0.099332
                     0.062865
                     0.009723
                     0.001947
                     0.162875
                     0.030179
                     0.000847
                     0.020789
                     0.008637
                     0.000203
                     0.010565
                     0